In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pandas_ods_reader import read_ods

In [2]:
#read file
#from pandas_ods_reader import read_ods
file = r'MasterFile - Year 2017.ods'
data = read_ods(file, 1)

In [3]:
data.head()

,ID,DMES,DCAL,G33L,CLG33L,DIFF,MR,05/08/17 – 15/02/28,unnamed.3
0,1.0,1.798717,1.778,0.971,0.04855,NaN,NaN,Gamma average,0.972400
1,2.0,1.779716,1.816,0.968,0.09695,0.003,NaN,STAN-DEV,0.000558
2,3.0,1.762920,1.786,0.999,0.14690,0.031,0.001789,β,6.000000
3,4.0,2.023142,1.991,0.961,0.19495,0.038,0.003789,T,1.000000
4,5.0,2.100162,2.069,0.998,0.24485,0.037,0.005737,Width AL,0.217966
